<a href="https://colab.research.google.com/github/Jakub-MFP/My_FIRE_Project/blob/master/portfolio_management/cashposition_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Objective**

## **Problem**

Lots of resources advocate for investors to have between 10% to 30% of their portfolio in cash so we can seize upon market opportunities. The problem is that it is never explained when we should have 30% or 20% or 10% cash position, what factors determine how much cash we should have at any given time?


## **Question**

What is the optimal cash position to keep in our portfolio at anytime for the past 20 years. We will assume that our portfolio only has one asset and it is the ETF SPY. Which is an ETF that tracks the SP500 Index and is a good overall indicator for the status of the "market"


## **Hypothesis**

I think we will figure out that just investing 100% of the money into a portfolio during our regular deposit cycles and keeping no cash position will under-perform having a cash position at all times. 

*For example*

* If the SPY is up, we keep 30% of our portfolio in cash
* IF SPY is down 10% from ATH (all time high) than we keep 20% of portfolio in cash
* IF SPY is down 20% from ATH than we keep 10% of portfolio in cash
* IF SPY is down 30% from ATH than we keep 5% of portfolio in cash 

I believe that having a cash position on the side and investing more when the market is down will yield a higher ROI and CAGR over the same 20 year period. 

If SPY just regular investments returned a 9% CAGR, than I believe doing something as simple as just holding some cash will return 12% CAGR+. Something significant like 30-40% higher returns than otherwise possible. 

This will be tested not just on SPY but a number of stocks and ETFs with the eventually goal of doing this  with multiple stocks in a portfolio and using the efficient frontier to determine how much we allocate to any stock when the market trigger is in effect to reduce or increase our cash position. 

## **The Procedure**
1. We run the experiment to see how much 10,000 in 2000 would yield by end of Dec 2019 with no additional investments. We want to calcualte the CAGR or Compound Annual Growth Rate
2. We run another experiment to see how much 10,000 in 2000 would yield by end of Dec 2019 with a 1,000 monthly deposit. Does it change the CAGR? 
3. Finally we will run a combinatorics trial 

   1. Start with 10,000
        1. Deposit each month 1,000 into portfolio
        2. Adjust the invested / cash ratio based on the market performance
            1. If the SPY is all time high than keep 70% invested and 30% in cash
            2. If the SPY drops 10% from ATM than keep 80% invested and 20% cash
            3. If SPY drops 20% from ATM than keep 90% invested and 10% cash
            4. If SPY drops 30% from ATM than keep 95% invested and 5% cash
        3. Do every combination of cash allocation based on how much market drops between -1% and -50%
        4. Calcualte the CAGR for each combination

Daily Progress Update 
https://myfireproject.com/topic/301-portfolio-management-research/


# **Initial Setup**

Installs Alpha Vantage and imports required modules

**Install Alpha Vantage API**

In [ ]:
# Install Alpha Vantage
!pip install alpha_vantage

**Import All The Modules You Need**

In [2]:
import pandas as pd
import json
import requests
import sqlite3
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt 
import sys
import math

from pandas import DataFrame
from datetime import datetime as dt
from alpha_vantage.timeseries import TimeSeries
from dateutil.relativedelta import relativedelta
from math import log

# **Alpaha Vantage API**

This will create a pandas dataframe of all historical prices

https://www.alphavantage.co/documentation/



Assign Stock Ticker and API Key

Get your free api key here > https://www.alphavantage.co/support/#api-key

In [3]:
stock_ticker = 'SPY'
   
    # Update this for your own API key
api_key = open('/content/drive/My Drive/Colab Notebooks/key').read()
#api_key = '4545sdsad5s4dsd'

Run API Request to get stock data

    # Date / Open / High / Low / Close / Adjusted Close / Volume / Dividend / Split

In [4]:
ts = TimeSeries (key=api_key, output_format = "pandas")
data_daily, meta_data = ts.get_daily_adjusted(symbol=stock_ticker, outputsize ='full')

#**Procedure**
Investment Period = Jan 1, 2000 to Dec 31, 2019
1. ***ROI*** and ***CAGR*** of Investing `$10,000` and just allocating `100%` Position
2. ***ROI*** and ***CAGR*** of Investing `$10,000` and depositing `$1,000` each month and allocating `100%` position 
3. ***ROI*** and ***CAGR*** of Investing `$10,000` and depositing `$1,000` each month and allocating a `variable` position using `market_status`
4. ***ROI*** and ***CAGR*** of Investing `$10,000` and depositing `$1,000` each month and allocating a `variable` position using `market_status` with `combinatorics`

## Step 1. `$10,000` and `100% Position` 

**Step 1 Overview**

You invest 10,000 on Jan 1, 2000 into the SPY. After 20 years you see what the value of your investment is and calcualte the CAGR. 

Edit
- `start_date = datetime.datetime(2000, 1, 1)` change the starting and end date in the formatting (YYYY, MM, DD)
-  `initial_investment` to whatever starting balance you want


In [ ]:
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2019, 12, 31)
time_difference = relativedelta(end_date, start_date)
number_of_years = float((time_difference.years)+1)

date_filter = data_daily[(data_daily.index > start_date) & (data_daily.index <= end_date)]

    # Starting Settings
initial_investment = 10000
monthly_deposit = 1000

    # Start Price
start_adjusted_close_price = date_filter['5. adjusted close'][-1]
starting_shares = initial_investment / start_adjusted_close_price

    # End Price

end_adjusted_close_price = date_filter['5. adjusted close'][0]


    # Final Math
step1_final_value = round((starting_shares * end_adjusted_close_price), 2)
step1_profit = round((step1_final_value - initial_investment), 2)
step1_roi = round((((step1_final_value - initial_investment) / initial_investment)* 100), 2)
step1_cagr = round((((step1_final_value / initial_investment) ** (1 / (number_of_years - 1)) - 1)* 100), 2)

    # Print out results

print("Start Share Price : ${} ".format(start_adjusted_close_price))
print("Starting Shares : {} ".format(starting_shares))
print("Number of years : {} ".format(number_of_years))
print("")

print("Ending Share Price : ${} ".format(end_adjusted_close_price))
print("")
print("Total Deposits : ${} ".format(initial_investment))
print("Portfolio Value : ${} ".format(step1_final_value))
print("Investment Profit : ${} ".format(step1_profit))
print("ROI : {}% ".format(step1_roi))
print("CAGR : {}% ".format(step1_cagr))

Start Share Price : $98.5115 
Starting Shares : 101.51099110256163 
Number of years : 20.0 

Ending Share Price : $317.2003 

Total Deposits : $10000 
Portfolio Value : $32199.32 
Investment Profit : $22199.32 
ROI : 221.99% 
CAGR : 6.35% 


## Step 2. `$10,000` + `$1,000` Monthly and `100% Position` 

This step investes 10,000 at the start, and than each new month we will add 1000 more to the portfolio. Whenever we add 1,000 to portfolio, we will also buy all the shares we can at current market price. 

In [7]:
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2019, 12, 31)
time_difference = relativedelta(end_date, start_date)
number_of_years = time_difference.years + 1

initial_investment = 10000
monthly_deposit = 1000


    # Create a filtered dataframe, and change the order it is displayed. 
date_filter = data_daily[(data_daily.index > start_date) & (data_daily.index <= end_date)]
date_filter = date_filter.sort_index(ascending=True)

    # Set starting balances
start_current_balance = initial_investment
start_adjusted_close_price = date_filter['5. adjusted close'][0]
start_current_shares = start_current_balance / start_adjusted_close_price

    # Settings for loop
current_balance = 0
current_shares = 0
deposit_month = 1
total_deposits_count = 0

    # Settings to create age of investments
transactions = [] # log all dates and transactions amounts
transactions.append([start_date, initial_investment])

    # add starting balance to deposit_dates_list
# deposit_dates.append(start_date, int(initial_investment))
#deposit_dates[start_date] = int(initial_investment)

    # step 4 - Itterate trough all the rows in the dataframe and buy shares if it is a new month
for index, row in date_filter.iterrows():

    current_date = str(index) # set current index date row to current_date 
    current_month = int(current_date[5:7]) # grab the current month segement from current date

    if current_month != deposit_month: # if it is a new month than run loop
        current_balance += monthly_deposit # add $1000 to current_balance
        shares_purchased = (current_balance / row['5. adjusted close'])
        current_shares += (current_balance / row['5. adjusted close']) # figure out how many new shares we are buying 
        total_deposits_count += 1 # add to total_deposits that we made a new deposit
        current_balance -= monthly_deposit # take of $1000 deposit for purchase of shares

        new_date = index
        new_transaction = monthly_deposit
        transactions.append([new_date, new_transaction]) #add current date for each deposit to the dic 
        
        deposit_month = current_month
        #print(deposit_month)
        #print(shares_purchased)
        

    # Figure out average age of deposits
age_list = []
for trans_date, deposit in transactions: # for items in list transactions
    calculation = (end_date - trans_date) #get difference in days
    age_years = float((int(calculation.days))/365) # convert to how many years
    age_weights = age_years * deposit #figure out the weight of each dollar invested
    age_list.append(age_weights) #add to list


age_ratio = sum(age_list)



    # Ending Metrics
end_adjusted_close_price = date_filter['5. adjusted close'][-1]
final_balance = round((current_shares * end_adjusted_close_price), 2)
total_deposits = int((total_deposits_count * monthly_deposit) + initial_investment)
investment_profit = final_balance - total_deposits
roi = round(((investment_profit / total_deposits) * 100), 2)
age = age_ratio / total_deposits
cagr = round((((final_balance / total_deposits) ** (1 / (age - 1)) - 1)* 100), 2)

Analysis of Step 2 Results

In [8]:
print("STARTING METRICS")
print("Initial Balance : $ {} ".format(start_current_balance))
print("Share Price : $ {} ".format(start_adjusted_close_price))
print("Purchased Shares : {} ".format(current_shares))

print("")
print("-----------------------------------------------------------")
print("")

print("ENDING METRICS")
print("Number of years : {} ".format(number_of_years))
print("Average Age of each Dollar invested : {} years ".format(age))
print("Share Price : $ {} ".format(end_adjusted_close_price))
print("Final Shares : {} ".format(current_shares))

print("")
print("-----------------------------------------------------------")
print("")

print("RESULTS")
print("Total Deposits : $ {} ".format(total_deposits))
print("Final Balance : $ {} ".format(final_balance))
print("Investment Profit : ${} ".format(investment_profit))
print("ROI : {}% ".format(roi))
print("CAGR : {}% ".format(cagr))

STARTING METRICS
Initial Balance : $ 10000 
Share Price : $ 98.5115 
Purchased Shares : 2183.797185730247 

-----------------------------------------------------------

ENDING METRICS
Number of years : 20 
Average Age of each Dollar invested : 10.405358419981294 years 
Share Price : $ 317.2003 
Final Shares : 2183.797185730247 

-----------------------------------------------------------

RESULTS
Total Deposits : $ 249000 
Final Balance : $ 692701.12 
Investment Profit : $443701.12 
ROI : 178.19% 
CAGR : 11.49% 


## Step 3. `$10,000` + `$1,000` Monthly and `Variable Position` 

1. Invest 10,000 at begining or x amount
2. Invest 1,000 each month or x amount
3. Determine what the market status is. Meaning how much the market has dropped from the all time high.
4. Based on how much the market has dropped, determine what % of the portfolio will be in SPY shares and what % will be in cash. 

The theory is that the more the market drops, the less in cash we should keep. 

In [5]:
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2019, 12, 31)
time_difference = relativedelta(end_date, start_date)
number_of_years = time_difference.years + 1

initial_investment = 10000
monthly_deposit = 1000


    # Create a filtered dataframe, and change the order it is displayed. 
date_filter = data_daily[(data_daily.index > start_date) & (data_daily.index <= end_date)]
date_filter = date_filter.sort_index(ascending=True)


    # Set starting balances
start_current_balance = initial_investment
start_adjusted_close_price = date_filter['5. adjusted close'][0]
start_current_shares = start_current_balance / start_adjusted_close_price

    # Settings for loop
current_balance = 0
portfolio_value = 0
current_shares = 0
deposit_month = 1
total_deposits_count = 0
total_shares_purchases_count = 0

    # Creating all the lists we need
transactions_deposits = [] #log every deposit in dictionary 
transactions_deposits.append([start_date, initial_investment])
transactions_trades = [] #log every trade in dictrionary
market_status_list = [] #log every market status for each day

    # settings for market status
market_all_time_high = 0 #defautl all time high is the first price of the share 
current_market_change = 0
current_market_status = 0

    # Main Loop
for index, row in date_filter.iterrows():

    current_date = str(index) # set current index date row to current_date 
    current_month = int(current_date[5:7]) # grab the current month segement from current date

    if current_month != deposit_month: # if it is a new month than run loop
        current_balance += monthly_deposit # add $1000 to current_balance
        total_deposits_count += 1 # add to total_deposits that we made a new deposit
        new_date = index
        new_transaction = monthly_deposit
        transactions_deposits.append([new_date, new_transaction]) #add current date for each deposit to the dic 
        deposit_month = current_month #set the deposit month to be current month


    # Figure out what the market status is, if it is down or up compared to all time high
    current_market_price = float(row['5. adjusted close']) #setting current market price to current row

    if (current_market_price > market_all_time_high): #checking if the current price is a new record for all time high
        market_all_time_high = current_market_price #if it is set the new marekt_all_time_high


    current_market_change = current_market_price - market_all_time_high # dollar value difference between all time high and current pric
    current_market_status = float(current_market_change / market_all_time_high) # % difference drop from all time high price
    market_status_list.append([current_date, current_market_price, market_all_time_high, current_market_change, current_market_status]) #logs market status into our list


        # How much cash do we need to kee on hand based on the market status
            # https://www.tutorialspoint.com/python/comparison_operators_example.htm
    if (current_market_status == 0):  # all time high record
        current_cash_required_equity = 0.10
    elif (current_market_status < -0.05): #less than 5%
        current_cash_required_equity = 0.9
    elif (current_market_status < -0.10): #less than 10%
        current_cash_required_equity = 0.8
    elif (current_market_status < -0.15): #less than 15%
        current_cash_required_equity = 0.07
    elif (current_market_status < -0.20): #less than 20%
        current_cash_required_equity = 0.06
    elif (current_market_status < -0.25): #less than 25%
        current_cash_required_equity = 0.05
    elif (current_market_status < -0.30): #less than 30%
        current_cash_required_equity = 0


    #print(current_cash_required_equity)

        # What is our current cash balance, and portfolio value
    portfolio_value = float((current_balance + (current_shares * current_market_price))) #figuring out the total value of portfolio
    current_cash_required = float(current_cash_required_equity * portfolio_value) #figuring ouit the dollar cash amount we need to keep in portoflio 

        # How many shares do we need to buy if any based on market status
    if (current_cash_required < current_balance): # checking if the current cash required is lower than our current balance. If it is we will need to buy shares to lower our position. 
        cash_to_purchase_shares = float(current_balance - current_cash_required) #how much cash we are going to use to buy shares
        current_shares += (cash_to_purchase_shares / current_market_price ) # figure out how many new shares we are buying and add it to our current shares
        current_balance -= (cash_to_purchase_shares) # deduct the cash we spent on buying shares from our current balance
        total_shares_purchases_count =+ 1 # add 1 to total count for cash trasncsstions
        transactions_trades.append([current_date, cash_to_purchase_shares, current_shares, current_balance]) #log all the transasctions
    #print(current_date, current_market_change)
    #print(current_date, current_market_price, market_all_time_high, current_market_change, current_market_status, current_cash_required_equity)

    # Figure out average age of deposits
age_list = []
for trans_date, deposit in transactions_deposits: # for items in list transactions
    calculation = (end_date - trans_date) #get difference in days
    age_years = float((int(calculation.days))/365) # convert to how many years
    age_weights = age_years * deposit #figure out the weight of each dollar invested
    age_list.append(age_weights) #add to list

age_ratio = sum(age_list)



    # Ending Metrics
end_adjusted_close_price = date_filter['5. adjusted close'][-1]
final_balance = round((current_shares * end_adjusted_close_price), 2)
total_deposits = int((total_deposits_count * monthly_deposit) + initial_investment)
investment_profit = final_balance - total_deposits
roi = round(((investment_profit / total_deposits) * 100), 2)
age = age_ratio / total_deposits
cagr = round((((final_balance / total_deposits) ** (1 / (age - 1)) - 1)* 100), 2)





Analysis of Step 3 Results

In [6]:
print("STARTING METRICS")
print("Initial Balance : $ {} ".format(start_current_balance))
print("Share Price : $ {} ".format(start_adjusted_close_price))
print("Purchased Shares : {} ".format(current_shares))

print("")
print("-----------------------------------------------------------")
print("")

print("ENDING METRICS")
print("Number of years : {} ".format(number_of_years))
print("Average Age of each Dollar invested : {} years ".format(age))
print("Share Price : $ {} ".format(end_adjusted_close_price))
print("Current Market Status: {}  ".format(current_market_status))
print("Current Cash Required: {}  ".format(current_cash_required))
print("Current Cash Required Equity: {}  ".format(current_cash_required_equity))


print("")
print("-----------------------------------------------------------")
print("")

print("TOTALS")
print("Total Deposits : $ {} ".format(total_deposits))
print("Final Portfolio Value : $ {} ".format(final_balance))
print("Final Shares : {} ".format(current_shares))
print("Final Cash : $ {} ".format(current_balance))

print("")
print("Investment Profit : ${} ".format(investment_profit))
print("ROI : {}% ".format(roi))
print("CAGR : {}% ".format(cagr))



STARTING METRICS
Initial Balance : $ 10000 
Share Price : $ 98.5115 
Purchased Shares : 1519.0531924959353 

-----------------------------------------------------------

ENDING METRICS
Number of years : 20 
Average Age of each Dollar invested : 10.405358419981294 years 
Share Price : $ 317.2003 
Current Market Status: -0.003344385978086724  
Current Cash Required: 53308.3902354657  
Current Cash Required Equity: 0.1  

-----------------------------------------------------------

TOTALS
Total Deposits : $ 249000 
Final Portfolio Value : $ 481844.13 
Final Shares : 1519.0531924959353 
Final Cash : $ 51239.77397898858 

Investment Profit : $232844.13 
ROI : 93.51% 
CAGR : 7.27% 


## Step 4. `$10,000` + `$1,000` Monthly and Solving `Variable Position` Combinatorically

The goal here is to do the same thing as in Step 3, but to go trough every single possible scenario in this loop.
```
    if (current_market_status > 0):  #greater than 0
        current_cash_required_equity = 0.3
    elif (current_market_status > -0.05): #less than 5%
        current_cash_required_equity = 0.25
    elif (current_market_status > -0.10): #less than 10%
        current_cash_required_equity = 0.20 
    elif (current_market_status > -0.15): #less than 15%
        current_cash_required_equity = 0.15
    elif (current_market_status > -0.20): #less than 20%
        current_cash_required_equity = 0.10
    elif (current_market_status > -0.25): #less than 25%
        current_cash_required_equity = 0.05
    elif (current_market_status > -0.30): #less than 30%
        current_cash_required_equity = 0
```

1. If market status > 0: than current_cash_required_equity = 0.00 to 0.50
2. If market status drops -0.01 to -0.50 for each option
    1. try current_cash_required_equity = 0.00 to 0.50

In [ ]:
loop_results = []    
cagr_list = []
    
status = np.arange(0, -0.3, -0.05)
req = np.arange(0.3, 0.00, -0.1)

start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2019, 12, 31)
time_difference = relativedelta(end_date, start_date)
number_of_years = time_difference.years + 1

initial_investment = 10000
monthly_deposit = 1000


# Create a filtered dataframe, and change the order it is displayed. 
date_filter = data_daily[(data_daily.index > start_date) & (data_daily.index <= end_date)]
date_filter = date_filter.sort_index(ascending=True)


# Set starting balances
start_current_balance = initial_investment
start_adjusted_close_price = date_filter['5. adjusted close'][0]
start_current_shares = start_current_balance / start_adjusted_close_price

max_cagr = 0

for first in range(len(req)):
    first_req = round(req[first], 2) # this is the 1st element, rounded to 2 digit
    for second in range(first, len(req)):
        second_req = round(req[second], 2) # same with 2nd element
        for third in range(second, len(req)):
            third_req = round(req[third], 2)
            for fourth in range(third, len(req)):
                fourth_req = round(req[fourth], 2)
                for fifth in range(fourth, len(req)):
                    fifth_req = round(req[fifth], 2)
                    for sixth in range(fifth, len(req)):
                        sixth_req = round(req[fifth], 2)
                        for seventh in range(sixth, len(req)):
                            seventh_req = round(req[sixth], 2)
                            for first in range(len(status)):
                                first_status = round(status[first], 2) 
                                for second in range(first, len(status)):
                                    second_status = round(status[second], 2) 
                                    for third in range(second, len(status)):
                                        third_status = round(status[third], 2)
                                        for fourth in range(third, len(status)):
                                            fourth_status = round(status[fourth], 2)
                                            for fifth in range(fourth, len(status)):
                                                fifth_status = round(status[fifth], 2)
                                                for sixth in range(fifth, len(status)):
                                                    sixth_status = round(status[fifth], 2)
                                                    for seventh in range(sixth, len(status)):
                                                        seventh_status = round(status[sixth], 2)


                                                        # Settings for loop
                                                        current_balance = 0
                                                        portfolio_value = 0
                                                        current_shares = 0
                                                        deposit_month = 1
                                                        total_deposits_count = 0
                                                        total_shares_purchases_count = 0

                                                        # Creating all the lists we need
                                                        transactions_deposits = [] #log every deposit in dictionary 
                                                        transactions_deposits.append([start_date, initial_investment])
                                                        transactions_trades = [] #log every trade in dictrionary
                                                        market_status_list = [] #log every market status for each day

                                                        # settings for market status
                                                        market_all_time_high = 0 #defautl all time high is the first price of the share 
                                                        current_market_change = 0
                                                        current_market_status = 0

                                                        # Main Loop
                                                        for index, row in date_filter.iterrows():

                                                            current_date = str(index) # set current index date row to current_date 
                                                            current_month = int(current_date[5:7]) # grab the current month segement from current date

                                                            if current_month != deposit_month: # if it is a new month than run loop
                                                                current_balance += monthly_deposit # add $1000 to current_balance
                                                                total_deposits_count += 1 # add to total_deposits that we made a new deposit
                                                                new_date = index
                                                                new_transaction = monthly_deposit
                                                                transactions_deposits.append([new_date, new_transaction]) #add current date for each deposit to the dic 
                                                                deposit_month = current_month #set the deposit month to be current month


                                                            # Figure out what the market status is, if it is down or up compared to all time high
                                                            current_market_price = float(row['5. adjusted close']) #setting current market price to current row

                                                            if (current_market_price > market_all_time_high): #checking if the current price is a new record for all time high
                                                                market_all_time_high = current_market_price #if it is set the new marekt_all_time_high


                                                            current_market_change = current_market_price - market_all_time_high # dollar value difference between all time high and current pric
                                                            current_market_status = float(current_market_change / market_all_time_high) # % difference drop from all time high price
                                                            #market_status_list.append([current_date, current_market_price, market_all_time_high, current_market_change, current_market_status]) #logs market status into our list


                                                                # How much cash do we need to kee on hand based on the market status
                                                                    # https://www.tutorialspoint.com/python/comparison_operators_example.htm
                                                            if (current_market_status == first_status):  # all time high record
                                                                current_cash_required_equity = first_req
                                                            elif (current_market_status < second_status): #less than 5%
                                                                current_cash_required_equity = second_req
                                                            elif (current_market_status < third_status): #less than 10%
                                                                current_cash_required_equity = third_req
                                                            elif (current_market_status < fourth_status): #less than 15%
                                                                current_cash_required_equity = fourth_req
                                                            elif (current_market_status < fifth_status): #less than 20%
                                                                current_cash_required_equity = fifth_req
                                                            elif (current_market_status < sixth_status): #less than 25%
                                                                current_cash_required_equity = sixth_req
                                                            elif (current_market_status < seventh_status): #less than 30%
                                                                current_cash_required_equity = seventh_req


                                                            #print(current_cash_required_equity)

                                                                # What is our current cash balance, and portfolio value
                                                            portfolio_value = float((current_balance + (current_shares * current_market_price))) #figuring out the total value of portfolio
                                                            current_cash_required = float(current_cash_required_equity * portfolio_value) #figuring ouit the dollar cash amount we need to keep in portoflio 

                                                                # How many shares do we need to buy if any based on market status
                                                            if (current_cash_required < current_balance): # checking if the current cash required is lower than our current balance. If it is we will need to buy shares to lower our position. 
                                                                cash_to_purchase_shares = float(current_balance - current_cash_required) #how much cash we are going to use to buy shares
                                                                current_shares += (cash_to_purchase_shares / current_market_price ) # figure out how many new shares we are buying and add it to our current shares
                                                                current_balance -= (cash_to_purchase_shares) # deduct the cash we spent on buying shares from our current balance
                                                                total_shares_purchases_count =+ 1 # add 1 to total count for cash trasncsstions
                                                                #transactions_trades.append([current_date, cash_to_purchase_shares, current_shares, current_balance]) #log all the transasctions
                                                            #print(current_date, current_market_change)
                                                            #print(current_date, current_market_price, market_all_time_high, current_market_change, current_market_status, current_cash_required_equity)

                                                        # Figure out average age of deposits
                                                        age_list = []
                                                        for trans_date, deposit in transactions_deposits: # for items in list transactions
                                                            calculation = (end_date - trans_date) #get difference in days
                                                            age_years = float((int(calculation.days))/365) # convert to how many years
                                                            age_weights = age_years * deposit #figure out the weight of each dollar invested
                                                            age_list.append(age_weights) #add to list

                                                        age_ratio = sum(age_list)



                                                            # Ending Metrics
                                                        end_adjusted_close_price = date_filter['5. adjusted close'][-1]
                                                        final_balance = round((current_shares * end_adjusted_close_price), 2)
                                                        total_deposits = int((total_deposits_count * monthly_deposit) + initial_investment)
                                                        investment_profit = final_balance - total_deposits
                                                        roi = round(((investment_profit / total_deposits) * 100), 2)
                                                        age = age_ratio / total_deposits
                                                        cagr = round((((final_balance / total_deposits) ** (1 / (age - 1)) - 1)* 100), 2)
                                                        #cagr_list.append(cagr)
                                                        if (cagr > max_cagr):
                                                            max_cagr = cagr
                                                        #max_cagr = max(cagr_list)
                                                        loop_results.append([final_balance, total_deposits, investment_profit, roi, age, cagr])
                                                        print("")
                                                        print("Market Status {} | {} | {} | {} | {} | {} | {}".format(first_status, second_status, third_status, fourth_status, fifth_status, sixth_status, seventh_status))
                                                        print("Cash Req {} | {} | {} | {} | {} | {} | {}".format(first_req, second_req, third_req, fourth_req, fifth_req, sixth_req, seventh_req))
                                                        print("Highest CAGR: {}".format(max_cagr))

# **Conclusion**

After setting the test conditions of just dollar cost avareging each month fully and getting 11.49% CAGR, I was unable to beat that performance using combinatorics. Atleast the way I had the combinatorics setup, the results didin't show any increase in ROI. Right now the best factor for identifying perfomance seems to be the lenght of time the money has been invested rather than specific times it was invested. 